In [5]:
import os
import pyodbc
import urllib
import sys
import pandas as pd
from pyspark import  SparkContext
from pyspark import SparkConf
from functools import reduce  # For Python 3.x
from pyspark.sql import SQLContext
from pyspark.sql import DataFrame
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import numpy as np
from glob import glob
from os.path import normpath, basename

print(sys.executable)

csv_path = '../data/extracts/v18/worldwide/105670/105670.csv'
path_to_save = '../data/extracts/v18/worldwide/105670/parquet'

catrader_schema = StructType([StructField("GeoId", LongType(), True), 
    StructField("PerilId", LongType(), True),
    StructField("LobId", LongType(), True),     
    StructField("EventId", LongType(), True),     
    StructField("Simulation", LongType(), True), 
    StructField("EventDay", LongType(), True),     
    StructField("Loss", DoubleType(), True)])

loss_schema = StructType([StructField("Simulation", LongType(), True), 
    StructField("EventId", LongType(), True),
    StructField("Day", LongType(), True),     
    StructField("Loss", DoubleType(), True),
    StructField("LayerAttr", DoubleType(), True),
    StructField("ReinsAttr", DoubleType(), True),
    StructField("CumLoss", DoubleType(), True),
    StructField("SimLoss", DoubleType(), True)])

def save_calc_csv_to_parquet(csv_path, path_to_save):
    customSchema = StructType([StructField("EventSetId", LongType(), True), 
    StructField("PerilId", LongType(), True), 
    StructField("GeoId", LongType(), True), 
    StructField("LobId", LongType(), False), 
    StructField("EventId", LongType(), True), 
    StructField("Simulation", LongType(), True), 
    StructField("EventDay", LongType(), True), 
    StructField("EventIsSynthetic", BooleanType(), True), 
    StructField("NthEvent", LongType(), True), 
    StructField("LayerAttrition", DoubleType(), True), 
    StructField("ReinstatementAttrition", DoubleType(), True),
    StructField("LayerAttritionByYear", DoubleType(), True), 
    StructField("ReinstatementAttritionByYear", DoubleType(), True)])
    
    df = sqlContext.read \
    .format('com.databricks.spark.csv') \
    .options(header='true') \
    .load(csv_path, schema = customSchema)
    
    df.write.parquet(path_to_save)
    
def catrader_csv_to_df(csv_path):
    df = sqlContext.read \
    .format('com.databricks.spark.csv') \
    .options(header='true') \
    .load(csv_path, schema = catrader_schema)
    return df

def save_to_parquet(df, path_to_save):
    df.write.parquet(path_to_save)
    
def save_to_parquet_with_partition(df, path_to_save):
    df.write.parquet(path_to_save)
                          


C:\ProgramData\Anaconda2\python.exe


In [6]:
sqlContext.setConf("spark.sql.parquet.compression.codec", "snappy")
df = catrader_csv_to_df(csv_path)    
save_to_parquet(df, path_to_save)

In [7]:
%%timeit -r 2

spark_df = sqlContext.read.load(path_to_save)
spark_df.groupBy().sum('Loss').show()

+--------------------+
|           sum(Loss)|
+--------------------+
|9.170692255465321E14|
+--------------------+

+--------------------+
|           sum(Loss)|
+--------------------+
|9.170692255465321E14|
+--------------------+

+--------------------+
|           sum(Loss)|
+--------------------+
|9.170692255465321E14|
+--------------------+

1 loop, best of 2: 890 ms per loop


In [9]:
#%%time
#should be faster and easier on the eye in spark 2.3
#https://databricks.com/blog/2017/10/30/introducing-vectorized-udfs-for-pyspark.html

def calculate(sim_block):
    sim = 0
    event_id = 1
    day = 2
    loss = 3
    cum_loss = 4
    sim_loss = 5
    
    return_rows = list()
    for row in sim_block:
            return_rows.append([row[sim], row[event_id], row[day], row[loss], 1.0, 0.0, row[cum_loss], row[sim_loss]])
    return return_rows

df = sqlContext.read.load(path_to_save, schema=catrader_schema)
df = df.repartition('Simulation')
df = df.groupBy('Simulation', 'EventId', 'EventDay').sum('Loss').withColumnRenamed('sum(Loss)', 'Loss')

w_prev = Window.partitionBy('Simulation').orderBy("EventDay").rowsBetween(Window.unboundedPreceding, Window.currentRow)
w_sim = Window.partitionBy('Simulation').orderBy("EventDay").rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

#df = df.where(df.Loss > 1000)
df = df.withColumn('CumLoss', f.sum('Loss').over(w_prev))
df = df.withColumn('SimLoss', f.sum('Loss').over(w_sim))

rdd = df.rdd.mapPartitions(calculate)
df = sqlContext.createDataFrame(rdd, loss_schema)
df.collect()
#save_to_parquet(df, path_to_save + 'xz-output-test')



MemoryError: 